This script downloads the entire season webpage from basketball-reference.com to a standings directory on your local machine. These HTML files will be parsed for individual box scores.

In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2016,2024)) # Define constant list of seasons

In [3]:
# Directory named data with 2 subdirectories: standings and scores
# Join paths
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
async def get_html(url, selector, sleep = 5, retries = 3):
    # Selector will be a CSS selector (ID that's used to locate unique element within the html)
    html = None
    for i in range(1, retries+1): # Increments more time to avoid sending to many requests and getting banned
        time.sleep(sleep * i)
        # Logic to handle errors when web scraping
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch() # Can also use p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    # Create links for each individual standings page
    soup = BeautifulSoup(html) # Create an instance of the BeautifulSoup class
    links = soup.find_all("a") # Finds all "a" (anchor) tags
    href = [l["href"] for l in links] # Grab href part of the anchor tag
    standings_pages = [f"https://basketball-reference.com{l}" for l in href] # Adds link to complete full URL
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path): # Check that we are not scraping data that has already been scraped
            continue
            
        html = await get_html(url, "#all_schedule") # Div element right outside of the table we are looking for
        with open(save_path, "w+") as f:
            f.write(html)

In [15]:
# Loop to download each season schedule page
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com


In [16]:
standings_files = os.listdir(STANDINGS_DIR)

In [17]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a") # Finds all "a" (anchor) tags
    hrefs = [l.get("href") for l in links] # Grab href part of the anchor tag

    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l] # Check for non-empty tag w/ "boxscore" and ".html"
    box_scores = [f"https://basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [18]:
standings_files = [s for s in standings_files if ".html" in s] # Checks for any non-html error files

In [19]:
# Loop to iterate through standings directory and scrape each individual box score page
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    await scrape_game(filepath)